### Basics
The fundamental idea is to compare a calculated value with expected value. For example, consider the following two operations:

In [1]:
// We need to test the following two functions:
function add(a, b){
    return a - b;
}

function subtract(a, b){
    return a - b;
}

// Testing add
let result = add(5, 3)
let expected = 8

if(result !== expected){
    throw new Error(`${result} is not equal to ${expected}`)
}

evalmachine.<anonymous>:17
    throw new Error(`${result} is not equal to ${expected}`);
    ^

Error: 2 is not equal to 8
    at evalmachine.<anonymous>:17:11
    at evalmachine.<anonymous>:20:3
    at sigintHandlersWrap (node:vm:271:12)
    at Script.runInContext (node:vm:139:14)
    at Object.runInContext (node:vm:292:6)
    at Object.execute (C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\executor.js:159:38)
    at JupyterHandlerImpl.handleExecuteImpl (C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\jupyter.js:206:38)
    at C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\jupyter.js:164:57
    at async JupyterHandlerImpl.handleExecute (C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\jupyter.js:164:21)
    at async ZmqServer.handleExecute (C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\jupyter.js:362:25)


In [ ]:
// Testing subtract
result = subtract(5, 3)
expected = 2

if(result !== expected){
    throw new Error(`${result} is not equal to ${expected}`)
}

The above code doesn't throw error indicating that it is working fine. We can abstract the logic as:

In [ ]:
function expect(actual){
    return {
        toBe: function(expected){
            if(actual !== expected){
                throw new Error(`${result} is not equal to ${expected}`)
            }
        }
    }
}

let result = subtract(6, 6)
let expected = 0

expect(result).toBe(expected)

We can have a number of different assertion methods in the returned object:

In [ ]:
function expect(actual){
    return {
        toBe: function(expected){
            if(actual !== expected){
                throw new Error(`${actual} is not equal to ${expected}`)
            }
        },
        
        toEqual: function(expected){},
        
        toBeGreaterThan: function(expected){}
    }
}

There are a couple of defeciencies in the above code. Firstly if multiple expect statements can potentially fail, our code block would fail only one and then stop. Second, we would need to dig into the code block to find out which one of the test case failed. We can improve this by introducing a test function:

In [5]:
function test(message, callback){
    try{
        callback()
        console.log(`✓ ${message}`)
    } catch(error) {
        console.log(`✕ ${message}`)
        console.log(error)
    }
}

// Use test to test out add and subtract
test('testing summation of two numbers', () => {
    let result = add(5, 6)
    let expected = 11
    
    expect(result).toBe(expected)
})

test('testing difference of two numbers', () => {
    let result = subtract(5, 6)
    let expected = -1
    
    expect(result).toBe(expected)
})

✕ testing summation of two numbers
Error: -1 is not equal to 11
    at Object.toBe (evalmachine.<anonymous>:6:23)
    at evalmachine.<anonymous>:33:20
    at test (evalmachine.<anonymous>:20:9)
    at evalmachine.<anonymous>:30:1
    at evalmachine.<anonymous>:41:3
    at sigintHandlersWrap (node:vm:271:12)
    at Script.runInContext (node:vm:139:14)
    at Object.runInContext (node:vm:292:6)
    at Object.execute (C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\executor.js:159:38)
    at JupyterHandlerImpl.handleExecuteImpl (C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\jupyter.js:206:38)
✓ testing difference of two numbers


What if we want to test asynchronous function? Consider the async equivalent of add and subtract:

In [6]:
function addAsync(a, b){
    return new Promise((resolve, reject) => {
        resolve(a - b)
    })
}

function subtractAsync(a, b){
    return new Promise((resolve, reject) => {
        resolve(a - b)
    })
}

// Use test to test out add and subtract
test('testing summation of two numbers', async () => {
    let result = await addAsync(5, 6)
    let expected = 11
    
    expect(result).toBe(expected)
})

test('testing difference of two numbers', async () => {
    let result = await subtractAsync(5, 6)
    let expected = -1
    
    expect(result).toBe(expected)
})

✓ testing summation of two numbers
✓ testing difference of two numbers


UnhandledPromiseRejection: Error: -1 is not equal to 11
    at Object.toBe (evalmachine.<anonymous>:6:23)
    at evalmachine.<anonymous>:29:20


We can see the problem in our code in the test function. The callback call would return a promise since callback here is an aync function. The fix is:

In [ ]:
async function test(message, callback){
    try{
        await callback()
        console.log(`✓ ${message}`)
    } catch(error) {
        console.log(`✕ ${message}`)
        console.log(error)
    }
}